In [83]:
include("../utils.jl")
data = getdata("../data/4")
using Dates

println(data[1:2])

function parsedata(d)
    c = match(r"\[(\d*)-(\d*)-(\d*) (.*):(.*)\] (.*)", d).captures
    date = map(x->parse(Int,x), c[1:end-1])
    date = DateTime(date[1], date[2], date[3], date[4], date[5])
    id = -1
    if c[end]=="falls asleep"
        action = -1
    elseif c[end]=="wakes up"
        action = 1
    else
        # Begins shift
        action = 0
        id = parse(Int, match(r".*#(\d*) .*", c[end]).captures[1])
    end
    return date, action, id
end

dates = map(x -> parsedata(x)[1], data)
actions = map(x -> parsedata(x)[2], data)
ids = map(x -> parsedata(x)[3], data)
println(dates[1:3])

p = sortperm(dates)
dates = dates[p]
actions = actions[p]
ids = ids[p];

["[1518-10-05 00:10] falls asleep", "[1518-07-22 23:53] Guard #1949 begins shift"]
DateTime[1518-10-05T00:10:00, 1518-07-22T23:53:00, 1518-07-06T00:55:00]


In [84]:
activeid = -1
startsleep = 0
maxid = maximum(ids)
asleep = Dict{Int, Dict{Int,Int}}()
for i in ids
    asleep[i] = Dict{Int,Int}()
end
for i=1:length(dates)
    if actions[i]==0
        activeid = ids[i]
        startsleep = 0
    elseif actions[i]==-1
        startsleep = Dates.minute(dates[i])
    else
        stopsleep = Dates.minute(dates[i])
        for minute = startsleep:stopsleep-1
            asleep[activeid][minute] = get(asleep[activeid], minute, 0) + 1
        end
    end
end

# Total minutes asleep
totalasleep = Dict{Int,Int}()
for i in ids
    if i!=-1
        totalasleep[i] = sum(values(asleep[i]))
    end
end
maxsleeper_id = findmax(totalasleep)[2]
maxminute = findmax(asleep[maxsleeper_id])[2]
println(maxsleeper_id*maxminute)

21083


In [85]:
maxasleep = Dict{Int,Int}()
for i in ids
    if i!=-1 && !isempty(asleep[i])
        maxasleep[i] = maximum(values(asleep[i]))
    end
end
maxsleeper_id = findmax(maxasleep)[2]
maxminute = findmax(asleep[maxsleeper_id])[2]
println(maxsleeper_id*maxminute)

53024
